In [1]:
class rayleigh_vapor:
    """Generates a vapor dataset from interpolated Rayleigh data"""
    def __init__(self,vdc_name=None,varnames=None,varfiles=None, rayleigh_root=None, 
                vapor_root=None, nxyz=None, grid_file=None, force=False, timeout=300,
                remove_means=[], rmins=[], rmaxes=[]):
        self.numts=len(varfiles)
        self.varnames=varnames
        self.vdc_name=vdc_name+'.vdc'
        self.vdc_data_dir = vdc_name+'_data'
        self.nvars=len(varnames)
        self.varfiles=varfiles
        self.timeout=timeout
        
        if (len(remove_means) != self.nvars):
            self.remove_mean=self.nvars*False
        else:
            self.remove_mean=remove_means
        
        if (len(rmins) != self.nvars):
            self.zero_rmin=False
            self.rmins=[None]*self.nvars
        else:
            self.zero_rmin=True
            self.rmins=rmins
            
        if (len(rmaxes) != self.nvars):
            self.zero_rmax=False
            self.rmaxes=[None]*self.nvars
        else:
            self.zero_rmax=True
            self.rmaxes=rmaxes       
        
        varstring=' '
        for i in range(self.nvars):
            varstring=varstring+varnames[i]
            if (i != self.nvars-1):
                varstring=varstring+':'
        self.varstring=varstring
        print(self.varstring)
        print(self.numts)
        self.vapor_root=vapor_root
        self.rayleigh_root=rayleigh_root
        self.nxyz=nxyz
        self.grid_file=grid_file
        if (force == True):
            print('Parameter "force" is set to true.')
            print('Removing: '+self.vdc_name+' > /dev/null')
            print('Removing: '+self.vdc_data_dir+' > /dev/null')
            self.destroy_vdc()            
    def create_vdc(self, force=False):
        import subprocess as sp
        res=str(self.nxyz)
        cube_string = res+'x'+res+'x'+res
        cmd1 = 'export PATH=$PATH:'+self.vapor_root+'/bin'
        cmd2 = ' && export LD_LIBRARY_PATH='+self.vapor_root+'/lib:$LD_LIBRARY_PATH'
        cmd3 = ' && vdccreate -dimension '+cube_string+' -numts '+str(self.numts)
        cmd3 = cmd3+' -vars3d '+self.varstring+' '+self.vdc_name
        creation_cmd=cmd1+cmd2+cmd3
        print(creation_cmd)
        s=sp.Popen(creation_cmd,shell=True)
        s.wait(timeout=self.timeout)

    def populate_vdc(self):
        for i in range(self.numts):
            for j in range(self.nvars):
                infile=self.varfiles[i][j]
                ofile=infile+'.cube'
                self.rayleigh_to_cube(infile,ofile,remove_mean=self.remove_mean[j], 
                                      rmin=self.rmins[j], rmax=self.rmaxes[j])
                self.cube_to_vdc(ofile,i,j)
    def rayleigh_to_cube(self,infile,ofile,remove_mean=False, rmin=None, rmax=None):
        import subprocess as sp
        cmd1 = 'export PATH=$PATH:'+self.rayleigh_root
        cmd2 = ' && echo $PATH && interp3d -i '+infile+' -o '+ofile+' -g '+self.grid_file+' -N '+str(self.nxyz)

        if(remove_mean):
            print('Removing mean.')
            cmd2=cmd2+" -rsm"

        if(rmin != None):
            print('Setting rmin')
            cmd2=cmd2+" -rmin "+str(rmin)
            print(cmd2)

        if(rmax != None):
            print('Setting rmax')
            cmd2=cmd2+" -rmax "+str(rmax)
            
        cmd = cmd1+cmd2
        s=sp.Popen(cmd,shell=True)
        s.wait(timeout=self.timeout)
        #b=s.read()
        #print(b)
    def cube_to_vdc(self,ofile,timeind,varind):
        import subprocess as sp
        cmd1 = 'export PATH=$PATH:'+self.vapor_root+'/bin'
        cmd2 = ' && export LD_LIBRARY_PATH='+self.vapor_root+'/lib:$LD_LIBRARY_PATH'
        cmd3 = ' && raw2vdc -ts '+str(timeind)+' -varname '+self.varnames[varind]
        cmd3 = cmd3+' '+self.vdc_name+' '+ofile
        cmd = cmd1+cmd2+cmd3
        s=sp.Popen(cmd, shell=True)
        s.wait(timeout=self.timeout)

    def destroy_vdc(self):
        import subprocess as sp
        cmd1 = 'rm -rf '+self.vdc_name +' > /dev/null'
        cmd2 = 'rm -rf '+self.vdc_data_dir+' > /dev/null'

        try:
            s=sp.Popen(cmd1,shell=True)
            s.wait(timeout=self.timeout)
            print(cmd1)
        except:
            print('cmd1 error', cmd1)
            s.communicate()
            pass
        #os.popen.wait()
        print(cmd2)
        try:
            s=sp.Popen(cmd2,shell=True)
            s.wait(timeout=self.timeout)
            print(cmd2)
        except:
            print('cmd2 error', cmd2)


def gen_3d_filelist( qcodes, diter, istart,iend, directory='Spherical_3D', ndig=8):
    files = []
    for i in range(istart,iend+diter,diter):
        fstring="{:0>"+str(ndig)+"d}"
        istring = directory+"/"+fstring.format(i)
        f = []
        for q in qcodes:
            qfnt="{:0>"+str(4)+"d}"
            qstr= qfnt.format(q)
            f.append(istring+'_'+qstr)
        files.append(f)
    return files

In [2]:
#Generate a list of files to use
qcodes = [501, 2807]
diter = 25  # time-step difference between outputs
first_iter = 25 # First time-step number to process
last_iter  = 200 # Last time-step number to process
#note -- this assumes that all files are in local directory Spherical_3D
#        set directory = 'X/Y/Z/Spherical_3D'  with no ending /  to change this
files=gen_3d_filelist(qcodes,diter,first_iter,last_iter)
print(files)

[['Spherical_3D/00000025_0501', 'Spherical_3D/00000025_2807'], ['Spherical_3D/00000050_0501', 'Spherical_3D/00000050_2807'], ['Spherical_3D/00000075_0501', 'Spherical_3D/00000075_2807'], ['Spherical_3D/00000100_0501', 'Spherical_3D/00000100_2807'], ['Spherical_3D/00000125_0501', 'Spherical_3D/00000125_2807'], ['Spherical_3D/00000150_0501', 'Spherical_3D/00000150_2807'], ['Spherical_3D/00000175_0501', 'Spherical_3D/00000175_2807'], ['Spherical_3D/00000200_0501', 'Spherical_3D/00000200_2807']]


In [3]:

gf = 'Spherical_3D/00000025_grid'

vdc = 'jup_test' #omit the .vdc name -- path is local (from where you ran jupyter notebook)
varnames = ['temp','vortz']
vroot='/custom/software/VAPOR3-3.2.0-Linux'   # base directory for Vapor
rroot='/home/feathern/devel/forks/Nick/Rayleigh/post_processing/interpolation' # where interp3d is located
cube_size=256
#If you would like to remove the spherical  mean, set hte remove_means keyword to
# an array of True/False values, 1 element per quantity code.
# If you want to remove means for no quantities, omit this keyword in the
# call to rayleigh_vapor alltogether.  If used, all values must be specified.
# False means that no action is taken for that particular quantity.  
# True means that the spherical mean is removed at each radius.
# In the example below, we remove the spherical mean for temperature, but
# not for vorticity.
remove_means=[True , False]
# If you would like to zero out r > rmax for each data set, provide an
# array of values below (1 array value for each quantity code).
# Optionally, don't pass the rmaxes keyword.  IF you want to zero out
# r > rmax for some quantities, but not for others, you still have to pass
# a full array, but set the non-zeroed elements to None.  In the example below,
# we zero out temperature for r > 1.0 and vorticity for r > 1.5
# This can be useful if boundary layers cause issues in your visualization.
rmaxes=[1.0, 1.5]  

#rmins works the same way.  In the example below, we perform no action on temperature,
# but for vorticity, we zero out r < 1.0
rmins=[None, 1.0]
test = rayleigh_vapor(vdc_name=vdc,varnames=varnames,varfiles=files,vapor_root=vroot,
                       nxyz=cube_size, grid_file=gf, rayleigh_root=rroot, force=True,
                       remove_means=remove_means, rmaxes=rmaxes, rmins=rmins)
test.create_vdc(force=True)

 temp:vortz
8
Parameter "force" is set to true.
Removing: jup_test.vdc > /dev/null
Removing: jup_test_data > /dev/null
rm -rf jup_test.vdc > /dev/null
rm -rf jup_test_data > /dev/null
rm -rf jup_test_data > /dev/null
export PATH=$PATH:/custom/software/VAPOR3-3.2.0-Linux/bin && export LD_LIBRARY_PATH=/custom/software/VAPOR3-3.2.0-Linux/lib:$LD_LIBRARY_PATH && vdccreate -dimension 256x256x256 -numts 8 -vars3d  temp:vortz jup_test.vdc


In [ ]:
test.populate_vdc()

Removing mean.
Setting rmax
Setting rmin
 && echo $PATH && interp3d -i Spherical_3D/00000025_2807 -o Spherical_3D/00000025_2807.cube -g Spherical_3D/00000025_grid -N 256 -rmin 1.0
Setting rmax
Removing mean.
Setting rmax
Setting rmin
 && echo $PATH && interp3d -i Spherical_3D/00000050_2807 -o Spherical_3D/00000050_2807.cube -g Spherical_3D/00000025_grid -N 256 -rmin 1.0
Setting rmax
Removing mean.
Setting rmax
Setting rmin
 && echo $PATH && interp3d -i Spherical_3D/00000075_2807 -o Spherical_3D/00000075_2807.cube -g Spherical_3D/00000025_grid -N 256 -rmin 1.0
Setting rmax
Removing mean.
Setting rmax
Setting rmin
 && echo $PATH && interp3d -i Spherical_3D/00000100_2807 -o Spherical_3D/00000100_2807.cube -g Spherical_3D/00000025_grid -N 256 -rmin 1.0
Setting rmax
Removing mean.
Setting rmax
Setting rmin
 && echo $PATH && interp3d -i Spherical_3D/00000125_2807 -o Spherical_3D/00000125_2807.cube -g Spherical_3D/00000025_grid -N 256 -rmin 1.0
Setting rmax
Removing mean.
Setting rmax
Setting